# Analysis of Mouse Brain Spatial Data

In [ ]:
import sctt as st
import scanpy as sc
import anndata
import scvelo as scv
import gseapy as gp
import numpy as np
import pandas as pd
from scipy.cluster.hierarchy import linkage, fcluster
import matplotlib as mpl
from adjustText import adjust_text

The processed data could be download [here](https://drive.google.com/file/d/1c53v1xo0wPd_2O7Zkfz1SWYsrv6e5bUC/view?usp=drive_link)

In [ ]:
data_dir = '../data/'
adata = sc.read_h5ad(data_dir+'mouse_brain.h5ad')

In [ ]:
adata

In [ ]:
sc.pp.neighbors(adata, use_rep = 'xy_loc',key_added ='spatial')

In [ ]:
sc.tl.leiden(adata,resolution = 0.3)

In [ ]:
sc.pl.scatter(adata, basis='xy_loc', color='leiden')

In [ ]:
adata.obs['leiden'].value_counts()

In [ ]:
U = adata.layers['unspliced']
S = adata.layers['spliced']
if 'toarray' in dir(U):
    U = U.toarray()
    S = S.toarray()
X_all = np.concatenate((U,S),axis = 1)
adata_aggr = anndata.AnnData(X=X_all)
sc.tl.pca(adata_aggr, svd_solver='arpack')
sc.pp.neighbors(adata_aggr)

In [ ]:
sc.tl.leiden(adata_aggr,resolution = 0.15)
leiden = adata_aggr.obs['leiden'].values

In [ ]:
adata_aggr.obsm['xy_loc'] = adata.obsm['xy_loc']
sc.pl.embedding(adata_aggr, basis='xy_loc', color='leiden')

In [ ]:
adata_aggr = st.dynamical_iteration(adata,n_states =8, n_iter = 15, return_aggr_obj=True, weight_connectivities = 0.5,n_components = 21, n_neighbors = 100,thresh_ms_gene = 0.2, use_spatial = True,spa_weight =0.1, thresh_entropy = 0.05)

In [ ]:
adata.obs['attractor'] = leiden

nrows =1
weights = [0.1,0.3,0.5,0.7,0.9]
ncols = len(weights)

fig, axes = plt.subplots(nrows, ncols, figsize=(14, 4))

for i,sw in enumerate(weights):
#adata.obsm['X_umap'] = adata_aggr.obsm['X_umap']
    adata_aggr = st.dynamical_iteration(adata,n_states =8, n_iter = 15, return_aggr_obj=True, weight_connectivities = 0.5,n_components = 21, n_neighbors = 100,thresh_ms_gene = 0.2, use_spatial = True,spa_weight = sw, thresh_entropy = 0.05)    
    ax = axes[i]
    sc.pl.embedding(adata, basis="xy_loc", color="attractor",show = False, ax = ax)
    ax.set_title('spatial_weight='+str(sw))
plt.tight_layout()
plt.show()

                            

In [ ]:
#adata_aggr.obsm['X_umap'] = adata.obsm['X_umap']
adata_aggr = st.dynamical_iteration(adata,n_states =8, n_iter = 15, return_aggr_obj=True, weight_connectivities = 0.5,n_components = 21, n_neighbors = 100,thresh_ms_gene = 0.2, use_spatial = True,spa_weight = 0.3, thresh_entropy = 0.05, stop_cr = 'rel')    

adata_aggr.obs['Region'] = adata.obs['Region'].values
sc.pl.embedding(adata, basis="xy_loc", color="attractor")

In [ ]:
st.plot_top_genes(adata, top_genes = 100, ncols =8, wspace = 1, hspace = 1, figsize = (12,20))

In [ ]:
st.plot_genes_list(adata, genelist = ['FOXP1','FTL','TGFBI'], figsize =(8,2) ,ncols =3, wspace = 0.9, hspace = 1)
#plt.savefig('./figures/emt_genes.pdf',dpi = 800,bbox_inches = 'tight')

In [ ]:
sc.set_figure_params(dpi=600,fontsize = 12)
mpl.rcParams['axes.grid'] = False
adata.obsm['X_xy_loc'] = adata.obsm['xy_loc']
adata_aggr.obsm['X_xy_loc']=adata.obsm['xy_loc']
adata_aggr.obsm['X_xy_loc_aggr']=adata.obsm['xy_loc']
adata.obsm['X_xy_loc_aggr']=adata.obsm['xy_loc']

st.plot_tensor(adata, adata_aggr, basis = 'xy_loc', list_attractor = [0,1,2],filter_cells = True, member_thresh = 0.1)

In [ ]:

st.plot_tensor(adata, adata_aggr, basis = 'xy_loc', list_attractor = [3,4,5],filter_cells = True, member_thresh = 0.1)

In [ ]:
st.plot_tensor(adata, adata_aggr, basis = 'xy_loc', list_attractor = [6,7],filter_cells = True, member_thresh = 0.1)

In [ ]:
sc.set_figure_params(dpi=600,fontsize = 12)
import matplotlib as mpl
mpl.rcParams['axes.grid'] = False
st.plot_tensor_single(adata, adata_aggr,  basis = 'xy_loc', state = 'joint',attractor = 2,filter_cells = True, member_thresh = 0.1)

In [ ]:
st.plot_tensor_single(adata, adata_aggr,  basis = 'xy_loc', state = 'joint',attractor = 1,filter_cells = True, member_thresh = 0.1)

In [ ]:
st.plot_tensor_single(adata, adata_aggr,  basis = 'umap', state = 'unspliced',attractor = 1,filter_cells = True, member_thresh = 0.2)

In [ ]:
st.plot_tensor_single(adata, adata_aggr,  basis = 'umap', state = 'unspliced',attractor = 2,filter_cells = True, member_thresh = 0.2)

In [ ]:
st.plot_tensor_single(adata, adata_aggr,  basis = 'xy_loc', state = 'joint',attractor = None,filter_cells = True, member_thresh = 0.2)

In [ ]:
st.plot_tensor_single(adata, adata_aggr,  basis = 'xy_loc', state = 'spliced',attractor = None,filter_cells = True, member_thresh = 0.2)

In [ ]:
st.plot_tensor_single(adata, adata_aggr,  basis = 'xy_loc', state = 'unspliced',attractor = None,filter_cells = True, member_thresh = 0.2)

In [ ]:
sc.pl.violin(adata, keys = ['entropy'], groupby='attractor')

In [ ]:
st.construct_landscape(adata, coord_key = 'xy_loc')

In [ ]:
sc.set_figure_params(dpi=600,figsize = (6,6) ,fontsize = 10)
st.infer_lineage(adata,si=0,sf=2, method = 'MPPT',flux_fraction=0.6)

In [ ]:
%matplotlib inline
sc.set_figure_params(dpi=600,figsize = (4,4) ,fontsize = 10)
st.plot_landscape(adata, dim = 3, elev=57, azim = 167,size_point = 8)

In [ ]:
ms_genes = adata.uns['gene_subset']
adata.uns['gene_subset'] = [x for x in ms_genes if x.startswith('Wnt')]
adata_aggr.uns['gene_subset'] = [x for x in ms_genes if x.startswith('Wnt')]

In [ ]:
st.plot_tensor(adata, adata_aggr, basis = 'xy_loc', list_attractor = [3,4,5],filter_cells = True, member_thresh = 0.1)

## Pathway Analysis

In [ ]:
pathway = gp.parser.download_library(name = 'KEGG_2019_Mouse')

In [ ]:
tpm_dict = {}
pathway_select = {}
temp = []
for key in pathway.keys():
    gene_list = [x.capitalize() for x in pathway[key]] 
    gene_select = [x for x in gene_list if x in adata_aggr.var_names]
    if len(gene_select)>2 and gene_select not in temp:
            scv.tl.velocity_graph(adata, vkey = 'vj', xkey = 'Ms', gene_subset = gene_select ,n_jobs = -1)
            tpm_dict[key] = adata.uns['vj_graph'].toarray().reshape(-1)
            pathway_select[key] = gene_select
            temp.append(gene_select)

In [ ]:
import numpy as np
import pandas as pd
import umap
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Create a pandas DataFrame from the dictionary
arr = np.stack(list(tpm_dict.values()))
cor = np.corrcoef(arr)

In [ ]:
pca = PCA(n_components=10)
pca_embedding = pca.fit_transform(cor)

# Perform UMAP on the PCA embedding
umap_reducer = umap.UMAP()
umap_embedding = umap_reducer.fit_transform(pca_embedding)

In [ ]:
# Perform hierarchical clustering
linked = linkage(cor, method='ward')

# Define the distance threshold or desired number of clusters
# Here, we'll use a distance threshold of 1.5
threshold = 1.5

# Get clustering labels as a NumPy array
c_labels = fcluster(linked, threshold, criterion='distance')

In [ ]:

# Sample data
x = umap_embedding[:, 0]
y = umap_embedding[:, 1]
labels = tpm_dict.keys()

# Create the scatter plot
fig, ax = plt.subplots()

num_clusters = max(c_labels)
cmap = plt.cm.get_cmap('Set2', num_clusters)

# Map the labels to colors using the colormap
colors = cmap((c_labels - 1) / (num_clusters - 1))

# Plot the scatter plot with colors based on the labels
sc = plt.scatter(x, y, c=colors)

# Remove the square outline
for spine in ['top','right']:
    ax.spines[spine].set_visible(False)


arrow_properties = dict(
    arrowstyle='->',  # Simple arrow with a head
    color='red',      # Arrow color
    linewidth=0.5,    # Arrow line width
    alpha=0.8,         # Arrow transparency     
    mutation_scale=5 
)
# Annotate points with labels
texts = []
for i, txt in enumerate(labels):
    texts.append(ax.annotate(txt, (x[i], y[i]), fontsize=8))
    
# Adjust the annotation positions to avoid overlaps
adjust_text(texts,arrowprops=arrow_properties)

# Hide axis ticks
ax.set_xticks([])
ax.set_yticks([])

# Hide axis labels (tick labels)
plt.xlabel('Embedding 1')
plt.ylabel('Embedding 2')

# Display the plot
plt.show()


In [ ]:
subset = pathway_select['TGF-beta signaling pathway']
adata.uns['gene_subset'] = subset
adata_aggr.uns['gene_subset'] = subset+[x+'_u' for x in subset]
st.plot_tensor_single(adata, adata_aggr, basis = 'xy_loc', state= 'joint')

In [ ]:
subset = pathway_select['Wnt signaling pathway']
adata.uns['gene_subset'] = subset
adata_aggr.uns['gene_subset'] = subset+[x+'_u' for x in subset]
st.plot_tensor_single(adata, adata_aggr, basis = 'xy_loc', state= 'joint')

In [ ]:
subset = pathway_select['MAPK signaling pathway']
adata.uns['gene_subset'] = subset
adata_aggr.uns['gene_subset'] = subset+[x+'_u' for x in subset]
st.plot_tensor_single(adata, adata_aggr, basis = 'xy_loc', state= 'joint')

In [ ]:
subset = pathway_select['Hippo signaling pathway']
adata.uns['gene_subset'] = subset
adata_aggr.uns['gene_subset'] = subset+[x+'_u' for x in subset]
st.plot_tensor_single(adata, adata_aggr, basis = 'xy_loc', state= 'joint')